# 目的与意义

## 背景

nifH 基因是研究生物固氮功能的重要标记基因。通过分析该基因的序列特征（如 GC 含量、序列长度等），可以评估其功能活跃性或丰度。这对于理解海洋生态系统中的氮循环具有重要意义。

## 研究目的

1. 探索 nifH 基因的关键特征与功能活跃性之间的关系。
2. 使用机器学习模型（线性回归、随机森林、神经网络），预测未知样本的固氮功能。
3. 比较不同模型的性能，评估其在生物信息学分析中的适用性。

## 意义

通过引入模型方法，可以：

1. 提高固氮基因功能预测的准确性，扩展其应用场景。
2. 捕捉复杂的特征关系，为未来基因功能研究提供参考。
3. 构建一个通用的分析框架，为其他基因功能分析提供方法论支持。


## ID：基因序列的唯一标识符。

## Sequence：基因的碱基序列（A、T、C、G）。

## Length：基因的长度，单位为碱基对。

## GC_Content：GC 含量，是 (G + C) / 总长度。


In [2]:
import sys
print(sys.version)

3.9.18 (main, Sep 11 2023, 08:38:23) 
[Clang 14.0.6 ]


In [3]:
from Bio import SeqIO
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score




<!-- 是一个典型的 nifH 基因序列长度范围，GC 含量为 0.646，符合固氮基因常见的高 GC 含量特性。 -->


In [4]:
# 定义主数据文件夹路径
data_folder = "./nifHdata"

# 存储所有序列的列表
all_sequences = []

# 标签，每个基因组的标签
all_labels = []

all_annotations = []  # 存储注释信息

In [5]:
# 遍历20个文件夹
# 遍历20个文件夹
for i in range(20):
    folder_name = f"nifH_datasets ({i})" if i > 0 else "nifH_datasets"
    folder_path = os.path.join(data_folder, folder_name)
    gene_file = os.path.join(folder_path, "ncbi_dataset/data/gene.fna")
    
    # 确保文件存在
    if os.path.exists(gene_file):
        # 读取 fasta 文件中的序列和注释信息
        for record in SeqIO.parse(gene_file, "fasta"):
            all_sequences.append(str(record.seq))  # 转换为字符串存储
            all_annotations.append(record.description)  # 存储注释信息
            all_labels.append(i)  # 标签对应文件夹编号
    else:
        print(f"can not find file：{gene_file}")


# 打印结果
print(f"successfully load {len(all_sequences)} 条序列！")
print(f"共有 {len(set(all_labels))} 个类别，标签为：{set(all_labels)}")
print("示例序列：", all_sequences[0][:50], "...")  # 只显示第一条序列前 50 个字符
print("示例注释：", all_annotations[0])  # 显示第一条序列的注释信息
print("示例标签：", all_labels[0])  # 显示第一条序列的标签


successfully load 20 条序列！
共有 20 个类别，标签为：{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
示例序列： ATGTCTTTGCGCCAGATTGCGTTCTACGGTAAGGGCGGTATCGGCAAGTC ...
示例注释： NZ_VISK01000015.1:262164-263045 nifH [organism=Azospirillum brasilense] [GeneID=56451760] [chromosome=]
示例标签： 0


In [6]:
print(all_sequences)

['ATGTCTTTGCGCCAGATTGCGTTCTACGGTAAGGGCGGTATCGGCAAGTCCACCACCTCCCAGAACACCCTGGCCGCGCTGGTCGAGCTGGATCAGAAGATCCTGATCGTCGGCTGCGATCCGAAGGCCGACTCGACCCGCCTGATCCTGCACGCCAAGGCGCAGGACACCGTGCTGCACCTCGCCGCCGAAGCCGGCTCGGTCGAGGATCTGGAGCTCGAGGACGTTCTCAAGATCGGCTACAAGGGCATCAAGTGCGTCGAGTCCGGCGGTCCGGAGCCGGGGGTCGGCTGCGCCGGCCGCGGCGTGATCACCTCGATCAACTTCCTGGAAGAGAACGGCGCCTACGACGACGTGGACTACGTCTCCTACGACGTGCTGGGCGACGTGGTGTGCGGCGGTTTCGCCATGCCCATCCGCGAGAACAAGGCCCAGGAAATCTACATCGTCATGTCCGGTGAGATGATGGCGCTCTACGCCGCCAACAACATCGCCAAGGGCATTCTGAAGTACGCCCACAGCGGCGGCGTGCGCCTCGGCGGCCTGATCTGCAACGAGCGCCAGACCGACAAGGAAATCGACCTCGCCTCCGCCCTGGCCGCCCGCCTCGGCACCCAGCTCATCCACTTCGTGCCGCGCGACAACATCGTGCAGCACGCCGAGCTGCGCCGCATGACGGTGATCGAGTACGCGCCGGACAGCCAGCAGGCCCAGGAATACCGCCAGCTCGCCAACAAGGTCCACGCGAACAAGGGCAAGGGCACCATCCCGACCCCGATCACGATGGAAGAGCTGGAAGAGATGCTGATGGACTTCGGCATCATGAAGTCGGAGGAGCAGCAGCTCGCCGAGCTCCAGGCCAAGGAAGCCGCCAAGGCCTGA', 'ATGAGACAGGTAGCAATATATGGAAAAGGCGGTATAGGAAAATCCACTACAACCCAGAACACGGTGGCAGCGTTGGCCGAAGCAGGAAAAAAGGTCATGGTGGTAGGATGCG

# Directly convert these original sequences into One-hot encoding

### The main reason for converting the original sequence into one-hot encoding is to represent the gene sequence in a numerical form that can be understood by computers while retaining biological information.

The original gene sequence is composed of characters (such as A, T, G, C, etc.), which cannot be processed directly by computers. Before modeling, these characters need to be converted into numerical form, and One-hot encoding is a common representation method that can retain the information of each base in the original sequence.


In [7]:
import numpy as np

# 定义 One-hot 编码函数
def one_hot_encode(seq, max_length):
    mapping = {'A': 0, 'T': 1, 'G': 2, 'C': 3, 'N': 4}  # 碱基映射
    one_hot = np.zeros((max_length, len(mapping)), dtype=np.float32)
    for i, char in enumerate(seq[:max_length]):  # 限制最大长度
        if char in mapping:
            one_hot[i, mapping[char]] = 1
    return one_hot

# Calculate the length of each sequence （计算每条序列的长度）


In [8]:

# 获取所有序列的最大长度
max_length = max(len(seq) for seq in all_sequences)

# 转换所有序列为 One-hot 编码
encoded_sequences = np.array([one_hot_encode(seq, max_length) for seq in all_sequences])
encoded_labels = np.array(all_labels, dtype=np.int32)

# 打印编码结果
print(f"One-hot 编码完成！总样本数：{encoded_sequences.shape[0]}")
print(f"每条序列的编码形状：{encoded_sequences.shape[1:]}")  # 例如 (max_length, 5)
print(f"标签形状：{encoded_labels.shape}")


One-hot 编码完成！总样本数：20
每条序列的编码形状：(894, 5)
标签形状：(20,)


In [9]:
from sklearn.model_selection import train_test_split

# 划分数据集
train_data, test_data, train_labels, test_labels = train_test_split(
    encoded_sequences, encoded_labels, test_size=0.2, random_state=42
)

# 打印数据划分结果
print(f"训练集样本数：{train_data.shape[0]}")
print(f"测试集样本数：{test_data.shape[0]}")
print(f"每条序列的形状：{train_data.shape[1:]}")
print(f"标签形状：训练集：{train_labels.shape}，测试集：{test_labels.shape}")


训练集样本数：16
测试集样本数：4
每条序列的形状：(894, 5)
标签形状：训练集：(16,)，测试集：(4,)


In [15]:
from Bio.Seq import Seq

# 送检序列
query_sequence = Seq("""
CTCCACTCGTCTGCTGCTCGGTGGACTGGCCCAGAAATCTGTACTTGATACTCTGCGGGAAGAAGGTGAGGACGTTGAACTCGACGATATCAGAAAAGCAGCTTACGGAGGAACCTGGGCAGTTGAATCAGGTGGCCCGGAGCCGGGTGTTGGCTGTGCAGGCCGAGGTATCATAACCGCGATTAACATGCTTGAATCCCTTGGCGCCTATGAGGAGAGCGAAAGCCTTGACTACGCCTTCTATGATGTTCTCGGTGATGTTGTTTGCGGTGGTTTTGCCATGCCCATCAGAGATGGTAAGGCGGAAGAAATCTATATCGTTGTCTCA
""".strip())

# 20 个样本序列
print(all_sequences)

# 最小匹配长度
min_length = 11

def count_long_repeats(query_sequence, reference_sequences, min_length=5):
    """
    统计送检序列与参考序列之间的匹配片段数量
    :param query_sequence: 送检序列
    :param reference_sequences: 参考序列列表
    :param min_length: 每次截取的长度（默认为 5）
    :return: 每个参考序列匹配片段的数量
    """
    # 结果存储每个样本的匹配数量
    match_counts = [0] * len(reference_sequences)

    # 遍历送检序列，以步长为 1 滑动窗口截取 min_length 长度的片段
    for i in range(len(query_sequence) - min_length + 1):
        query_fragment = str(query_sequence[i:i + min_length])  # 将片段转换为字符串
        
        # 对每个样本序列进行比对
        for idx, reference_sequence in enumerate(reference_sequences):
            if query_fragment in reference_sequence:
                match_counts[idx] += 1

    return match_counts

# 调用函数计算匹配数量
match_counts = count_long_repeats(query_sequence, all_sequences, min_length=min_length)

# 打印结果
for idx, count in enumerate(match_counts):
    print(f"样本 {idx} 与送检序列的长连续匹配片段数量：{count}")

['ATGTCTTTGCGCCAGATTGCGTTCTACGGTAAGGGCGGTATCGGCAAGTCCACCACCTCCCAGAACACCCTGGCCGCGCTGGTCGAGCTGGATCAGAAGATCCTGATCGTCGGCTGCGATCCGAAGGCCGACTCGACCCGCCTGATCCTGCACGCCAAGGCGCAGGACACCGTGCTGCACCTCGCCGCCGAAGCCGGCTCGGTCGAGGATCTGGAGCTCGAGGACGTTCTCAAGATCGGCTACAAGGGCATCAAGTGCGTCGAGTCCGGCGGTCCGGAGCCGGGGGTCGGCTGCGCCGGCCGCGGCGTGATCACCTCGATCAACTTCCTGGAAGAGAACGGCGCCTACGACGACGTGGACTACGTCTCCTACGACGTGCTGGGCGACGTGGTGTGCGGCGGTTTCGCCATGCCCATCCGCGAGAACAAGGCCCAGGAAATCTACATCGTCATGTCCGGTGAGATGATGGCGCTCTACGCCGCCAACAACATCGCCAAGGGCATTCTGAAGTACGCCCACAGCGGCGGCGTGCGCCTCGGCGGCCTGATCTGCAACGAGCGCCAGACCGACAAGGAAATCGACCTCGCCTCCGCCCTGGCCGCCCGCCTCGGCACCCAGCTCATCCACTTCGTGCCGCGCGACAACATCGTGCAGCACGCCGAGCTGCGCCGCATGACGGTGATCGAGTACGCGCCGGACAGCCAGCAGGCCCAGGAATACCGCCAGCTCGCCAACAAGGTCCACGCGAACAAGGGCAAGGGCACCATCCCGACCCCGATCACGATGGAAGAGCTGGAAGAGATGCTGATGGACTTCGGCATCATGAAGTCGGAGGAGCAGCAGCTCGCCGAGCTCCAGGCCAAGGAAGCCGCCAAGGCCTGA', 'ATGAGACAGGTAGCAATATATGGAAAAGGCGGTATAGGAAAATCCACTACAACCCAGAACACGGTGGCAGCGTTGGCCGAAGCAGGAAAAAAGGTCATGGTGGTAGGATGCG

In [129]:
from Bio import SeqIO
from Bio import pairwise2
from scipy.stats import entropy
import numpy as np
import pandas as pd
from itertools import groupby

# 计算特征函数
def calculate_features(sequence):
    # ATCG 所占比例
    length = len(sequence)
    A_ratio = sequence.count('A') / length
    T_ratio = sequence.count('T') / length
    C_ratio = sequence.count('C') / length
    G_ratio = sequence.count('G') / length

    # GC含量
    gc_content = (sequence.count('G') + sequence.count('C')) / length

    # 大于 4 个碱基长度的连续重复序列数
    long_repeats_count = sum(1 for i in range(len(sequence) - 4) if sequence[i:i + 5] == sequence[i] * 5)

    # 序列熵（香农熵）
    base_counts = [sequence.count(base) / length for base in "ATCG"]
    seq_entropy = entropy(base_counts)

    # 最长碱基重复长度
    max_repeat_length = max(len(list(g)) for _, g in groupby(sequence))

    return {
        "A_Ratio": A_ratio,
        "T_Ratio": T_ratio,
        "C_Ratio": C_ratio,
        "G_Ratio": G_ratio,
        "GC_Content": gc_content,
        "Long_Repeats_Count": long_repeats_count,
        "Sequence_Entropy": seq_entropy,
        "Max_Repeat_Length": max_repeat_length
    }

# 示例计算
all_sequences = [
    # 假设你有一些 fasta 格式的样本，这里简单模拟了序列
    "ATGCGTACGTAGCTAGCTAGCTA",
    "GCGCGCGTGTGTATATATAGCGC"
]

# 计算每个序列的特征
features = [calculate_features(seq) for seq in all_sequences]

# 转换为数据框便于查看
features_df = pd.DataFrame(features)
print(features_df)


    A_Ratio  T_Ratio   C_Ratio   G_Ratio  GC_Content  Long_Repeats_Count  \
0  0.260870  0.26087  0.217391  0.260870    0.478261                   0   
1  0.173913  0.26087  0.217391  0.347826    0.565217                   0   

   Sequence_Entropy  Max_Repeat_Length  
0          1.383370                  1  
1          1.353822                  1  
